In [4]:
from os import listdir
from os.path import isfile, join
import io
from zipfile import ZipFile
import requests
import pandas as pd
from datetime import datetime
from IPython.display import display, Markdown, Latex

import geopy.distance

import json

In [2]:
# Base URL de la api de Ingenier@
IGA_API_BASE_URL = "http://ing-acc-movil01.personal.corp/ingenier@/symfony/public/index.php/api"
JOB_SRC_DATA_FOLDER = '..\\src_data\\traffica_insights_jobs\\1'
MIN_RSRP_DBM = -95
NEAR_SAMPLES_PERCENT = 90.0

---
# Traffica Insights Data

---

In [17]:
def extract_zip(input_zip):
    input_zip=ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}

def extract_job_src_filenames(folder: str):
    raw = None
    post_processed = None
    files = [f for f in listdir(folder) if isfile(join(folder, f))]
    for file in files:
        if not file.endswith('.zip'):
            continue
        if file.startswith('traffica_insights_job'):
            raw = file
        if file.startswith('traffica_insights_post_processing_job'):
            post_processed = file
    return {
        'raw': raw,
        'post_processed': post_processed,
    }

filenames = extract_job_src_filenames(JOB_SRC_DATA_FOLDER)
raw_filename = filenames['raw']
post_processed_filename = filenames['post_processed']

# Job
df_job_data = None
if raw_filename is not None:
    path = JOB_SRC_DATA_FOLDER + '\\' + raw_filename
    data = extract_zip(path)
    
    df_job_data = pd.read_csv(io.BytesIO(next(iter(data.values()))), sep=';', decimal=',')
    
    display(Markdown('## ' + raw_filename))
    display(Markdown('### Datos Crudos'))
    display(df_job_data)


# Post-Processing Job
df_stats_summary = None
df_stats_total = None
df_stats_filtered = None
df_post_proc_job_data = None
if post_processed_filename is not None:
    path = JOB_SRC_DATA_FOLDER + '\\' + post_processed_filename
    data = extract_zip(path)

    for key in data.keys():
        if key.startswith('stats'):
            df_stats_summary = pd.read_excel(io.BytesIO(data[key]), sheet_name='Resumen')
            df_stats_total = pd.read_excel(io.BytesIO(data[key]), sheet_name='Detalle (Total)')
            df_stats_filtered = pd.read_excel(io.BytesIO(data[key]), sheet_name='Detalle (Filtrado)')
        if key.startswith('data'):
            df_post_proc_job_data = pd.read_csv(io.BytesIO(data[key]))

    display(Markdown('## ' + post_processed_filename))
    display(Markdown('### Resumen'))
    display(df_stats_summary)
    display(Markdown('### Detalle (Total)'))
    display(df_stats_total)
    display(Markdown('### Detalle (Filtrado)'))
    display(df_stats_filtered)
    display(Markdown('### Datos Postprocesados'))
    display(df_post_proc_job_data)

## traffica_insights_job_241.zip

### Datos Crudos

,﻿Database,Event,Datetime,SiteName,EMG,SectorName,TechBand,Report Time,Mobile Station,PLMN Id,...,DL Bytes Total GBR,DL Mean WB CQI,UL Bytes Total Non GBR,UL Bytes Total GBR,UL Mean PUSCH RSSI,UL Mean PUCCH RSSI,UE UL Throughput,GPS Confidence,Scenario,Final TA Adjusted
0,PWTRAFFICATNES1,ENODEB ACTIVE RTT,2024-07-19 08:00:00,A07-S012,ZSJRIN,ZSJRINL13,LTE FDD_AWS_2225,2024-07-19T08:00:00.120,NaN,72234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1521.0
1,PWTRAFFICATNES1,ENODEB ACTIVE RTT,2024-07-19 08:00:01,A07-S012,ZSJRIN,ZSJRINF12,LTE FDD_2600MHz_3250,2024-07-19T08:00:01.340,NaN,72234,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2379.0
2,PWTRAFFICATNES1,ENODEB ACTIVE RTT,2024-07-19 08:00:01,A07-S012,ZSJRIN,ZSJRINF12,LTE FDD_2600MHz_3250,2024-07-19T08:00:01.910,NaN,72234,...,0.0,12.0,827.0,0.0,-93.0,-108.0,315.0,NaN,NaN,1053.0
3,PWTRAFFICATNES1,ENODEB ACTIVE RTT,2024-07-19 08:00:02,A07-S012,ZSJRIN,ZSJRINL13,LTE FDD_AWS_2225,2024-07-19T08:00:02.510,NaN,72234,...,0.0,9.0,2645.0,0.0,-118.0,-111.0,25.0,NaN,NaN,1365.0
4,PWTRAFFICATNES1,ENODEB ACTIVE RTT,2024-07-19 08:00:09,A07-S012,ZSJRIN,ZSJRINN13,LTE FDD_700MHz_9360,2024-07-19T08:00:09.650,NaN,72234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798038,PWTRAFFICATNES2,ENODEB ACTIVE RTT,2024-07-19 10:44:59,A07-S044,ZSTOME,ZSTOMEM15,LTE FDD_1900MHz_976,2024-07-19T10:44:59.670,NaN,72234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,429.0
798039,PWTRAFFICATNES2,ENODEB ACTIVE RTT,2024-07-19 10:45:00,A07-S044,ZSTOME,ZSTOMEM15,LTE FDD_1900MHz_976,2024-07-19T10:45:00.940,NaN,72234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4485.0
798040,PWTRAFFICATNES2,ENODEB ACTIVE RTT,2024-07-19 10:46:16,A07-S044,ZSTOME,ZSTOMEM16,LTE FDD_1900MHz_976,2024-07-19T10:46:16.760,NaN,72234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,195.0
798041,PWTRAFFICATNES2,ENODEB ACTIVE RTT,2024-07-19 10:46:21,A07-S044,ZSTOME,ZSTOMEM15,LTE FDD_1900MHz_976,2024-07-19T10:46:21.710,NaN,72234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,351.0


## traffica_insights_post_processing_job_89.zip

### Resumen

...

,Cód. Sitio,EMG,eNodeB Id,Inicio Consulta,Final Consulta,Registros Totales,Complet. Datos [%],Distrib. de Registros
0,A07-S012,ZSJRIN,70012,2024-07-19 08:00:00,2024-07-19 22:00:00,15430,99.998016,▒▒▒▒▒▒▓░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▒░░░░░░░░░░░░░...
1,A07-S028,ZSVIEJ,70028,2024-07-19 08:00:00,2024-07-19 22:00:00,4267,95.236111,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
2,A07-S043,ZSFCEN,70043,2024-07-19 08:00:00,2024-07-19 22:00:00,21183,99.998016,▓▓▓▒▓▓░░░░░░░░░░░░░░░░▒██▓░░░░░░░░▒▓▓▓▒░░░░░░░...
3,A07-S044,ZSTOME,70044,2024-07-19 08:00:00,2024-07-19 22:00:00,5201,95.232143,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
4,A07-S047,ZSFCOL,70047,2024-07-19 08:00:00,2024-07-19 22:00:00,15000,94.148810,▒▒▒▒▒▒▒▓▒░░░░░░░░░░░░░░░░░░░░░░░░▒▓▓▓▓▓▒░░░░░░...
...,...,...,...,...,...,...,...,...
111,A07-S377,ZSFNUN,73377,2024-07-19 08:00:00,2024-07-19 22:00:00,259,19.047619,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
112,A07-S382,ZSTALB,73382,2024-07-19 08:00:00,2024-07-19 22:00:00,209,19.011905,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
113,A07-S383,ZSTMAR,73383,2024-07-19 08:00:00,2024-07-19 22:00:00,361,18.821429,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
114,A07-S390,ZSFCGO,73390,2024-07-19 08:00:00,2024-07-19 22:00:00,524,37.914683,▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░...


### Detalle (Filtrado)

,Cód. Sitio,EMG,eNodeB Id,Inicio Consulta,Final Consulta,Registros Totales,Complet. Datos [%],Distrib. de Registros
0,A07-S012,ZSJRIN,70012,2024-07-19 08:00:00,2024-07-19 22:00:00,15430,99.998016,▒▒▒▒▒▒▓░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▒░░░░░░░░░░░░░...
1,A07-S028,ZSVIEJ,70028,2024-07-19 08:00:00,2024-07-19 22:00:00,4267,95.236111,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
2,A07-S043,ZSFCEN,70043,2024-07-19 08:00:00,2024-07-19 22:00:00,21183,99.998016,▓▓▓▒▓▓░░░░░░░░░░░░░░░░▒██▓░░░░░░░░▒▓▓▓▒░░░░░░░...
3,A07-S044,ZSTOME,70044,2024-07-19 08:00:00,2024-07-19 22:00:00,5201,95.232143,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
4,A07-S047,ZSFCOL,70047,2024-07-19 08:00:00,2024-07-19 22:00:00,15000,94.148810,▒▒▒▒▒▒▒▓▒░░░░░░░░░░░░░░░░░░░░░░░░▒▓▓▓▓▓▒░░░░░░...
...,...,...,...,...,...,...,...,...
111,A07-S377,ZSFNUN,73377,2024-07-19 08:00:00,2024-07-19 22:00:00,259,19.047619,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
112,A07-S382,ZSTALB,73382,2024-07-19 08:00:00,2024-07-19 22:00:00,209,19.011905,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
113,A07-S383,ZSTMAR,73383,2024-07-19 08:00:00,2024-07-19 22:00:00,361,18.821429,▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
114,A07-S390,ZSFCGO,73390,2024-07-19 08:00:00,2024-07-19 22:00:00,524,37.914683,▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░...


### Datos Postprocesados

,Latitud,Longitud,Reg. Totales,Celda Dom. -> Nombre,Celda Dom. -> Reg. Totales,Celda Dom. -> RSRP (Avg),Best Serv. -> Nombre,Best Serv. -> Reg. Totales,Best Serv. -> RSRP (Avg),RSRP Measurement Report Source1 (Avg),...,DL Mean WB CQI (Avg),UL Mean PUSCH RSSI (Avg),UL Mean PUCCH RSSI (Avg),DL Bytes Total Non GBR (Sum),DL Bytes Total GBR (Sum),UL Bytes Total Non GBR (Sum),UL Bytes Total GBR (Sum),UL+DL Bytes Total Non GBR (Sum),UL+DL Bytes Total GBR (Sum),UL+DL Bytes Total (Sum)
0,-31.613620,-60.585502,1,ZSJRINL13,1,-110.000000,ZSJRINL13,1,-110.000000,-110.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
1,-31.623332,-60.588249,7,ZSJRINN12,3,-103.666667,ZSJRINL12,2,-97.500000,-104.000000,...,7.666667,-101.000000,-106.000000,969208,0,26544,0,995752,0,995752
2,-31.609663,-60.567756,391,ZSJRINF12,343,-100.253644,ZSJRINM12,14,-95.357143,-100.025575,...,12.839009,-100.337243,-107.848214,8773172,0,2721796,0,11494968,0,11494968
3,-31.613620,-60.568601,40,ZCOLASN11,23,-93.217391,ZSFMAKF11,2,-92.000000,-98.325000,...,11.645161,-101.593750,-102.300000,6904060,0,432168,0,7336228,0,7336228
4,-31.629448,-60.679095,7,ZSFPUEN11,3,-110.666667,ZSJRINF13,1,-107.000000,-110.142857,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122898,-31.658406,-60.841349,1,ZSTOMEM15,1,-55.000000,ZSTOMEM15,1,-55.000000,-55.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
122899,-31.658406,-60.841772,2,ZSTOMEM16,1,-43.000000,ZSTOMEM16,1,-43.000000,-51.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
122900,-31.658226,-60.841983,1,ZSTOMEM15,1,-45.000000,ZSTOMEM15,1,-45.000000,-45.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
122901,-31.658766,-60.841560,1,ZSTOMEM16,1,-74.000000,ZSTOMEM16,1,-74.000000,-74.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0


In [18]:
df_ti_raw = None
df_ti_best_serv_cell = None
df_ti_dominant_cell = None

# Raw
if df_job_data is not None:
    df_ti_raw = df_job_data[['SectorName', 'Latitude', 'Longitude', 'RSRP Measurement Report Source1']].rename(columns={ 
        'SectorName': 'sector_name', 
        'Latitude': 'lat', 
        'Longitude': 'lng', 
        'RSRP Measurement Report Source1': 'rsrp',
    }).set_index('sector_name')
    df_ti_raw = df_ti_raw.assign(total_samples = lambda r: 1)
    display(Markdown('### Muestras Crudo'))
    display(df_ti_raw)

# Post-processed
if df_post_proc_job_data is not None:
    df_ti_dominant_cell = df_post_proc_job_data[['Celda Dom. -> Nombre', 'Latitud', 'Longitud', 'Celda Dom. -> RSRP (Avg)', 'Celda Dom. -> Reg. Totales']].rename(columns={ 
        'Celda Dom. -> Nombre': 'sector_name', 
        'Latitud': 'lat', 
        'Longitud': 'lng', 
        'Celda Dom. -> RSRP (Avg)': 'rsrp',
        'Celda Dom. -> Reg. Totales': 'total_samples',
    }).set_index('sector_name')
    display(Markdown('### Muestras Dominant Cell'))
    display(df_ti_dominant_cell)

    df_ti_best_serv_cell = df_post_proc_job_data[['Best Serv. -> Nombre', 'Latitud', 'Longitud', 'Best Serv. -> RSRP (Avg)', 'Best Serv. -> Reg. Totales']].rename(columns={ 
        'Best Serv. -> Nombre': 'sector_name', 
        'Latitud': 'lat', 
        'Longitud': 'lng', 
        'Best Serv. -> RSRP (Avg)': 'rsrp',
        'Best Serv. -> Reg. Totales': 'total_samples',
    }).set_index('sector_name')
    display(Markdown('### Muestras Best Server'))
    display(df_ti_best_serv_cell)

### Muestras Crudo

,lat,lng,rsrp,total_samples
sector_name,,,,
ZSJRINL13,-31.613706,-60.585415,-110,1
ZSJRINF12,-31.623352,-60.588291,-112,1
ZSJRINF12,-31.609747,-60.567691,-100,1
ZSJRINL13,-31.613599,-60.568550,-109,1
ZSJRINN13,-31.629435,-60.679014,-112,1
...,...,...,...,...
ZSTOMEM15,-31.658628,-60.845504,-86,1
ZSTOMEM15,-31.660827,-60.795722,-107,1
ZSTOMEM16,-31.658757,-60.841599,-74,1


### Muestras Dominant Cell

,lat,lng,rsrp,total_samples
sector_name,,,,
ZSJRINL13,-31.613620,-60.585502,-110.000000,1
ZSJRINN12,-31.623332,-60.588249,-103.666667,3
ZSJRINF12,-31.609663,-60.567756,-100.253644,343
ZCOLASN11,-31.613620,-60.568601,-93.217391,23
ZSFPUEN11,-31.629448,-60.679095,-110.666667,3
...,...,...,...,...
ZSTOMEM15,-31.658406,-60.841349,-55.000000,1
ZSTOMEM16,-31.658406,-60.841772,-43.000000,1
ZSTOMEM15,-31.658226,-60.841983,-45.000000,1


### Muestras Best Server

,lat,lng,rsrp,total_samples
sector_name,,,,
ZSJRINL13,-31.613620,-60.585502,-110.000000,1
ZSJRINL12,-31.623332,-60.588249,-97.500000,2
ZSJRINM12,-31.609663,-60.567756,-95.357143,14
ZSFMAKF11,-31.613620,-60.568601,-92.000000,2
ZSJRINF13,-31.629448,-60.679095,-107.000000,1
...,...,...,...,...
ZSTOMEM15,-31.658406,-60.841349,-55.000000,1
ZSTOMEM16,-31.658406,-60.841772,-43.000000,1
ZSTOMEM15,-31.658226,-60.841983,-45.000000,1


---
# Radios Aproximados de Sectores

---

In [13]:
payload = {
    "reportName": None,
    "siteNames": [],
    "filteringParams": {"SiteVisualizationData.SiteStatus": ["OA"]},
    "selectBySiteNames": False,
    "selectByFilteringParams": True
}

payload['reportName'] = "datos_radio_sector"
x = requests.post(IGA_API_BASE_URL + '/public/report/generic-report', json=payload)
with io.BytesIO(x.content) as fh:
    df_radio_sector_raw = pd.read_excel(fh, "Radios aproximados de sectores")

payload['reportName'] = "datos_outdoor"
x = requests.post(IGA_API_BASE_URL + '/public/report/generic-report', json=payload)
with io.BytesIO(x.content) as fh:
    df_outdoor_raw = pd.read_excel(fh, "Datos outdoor")

In [14]:
df_iga_1 = ((
    df_outdoor_raw[['Código', 'Sector', 'Estado', 'Latitud Antena', 'Longitud Antena', 'Azimuth Efectivo', 'Ancho Horiz.']]
).rename(columns = {
    'Código': 'site_name', 
    'Sector': 'sector_name', 
    'Estado': 'site_status', 
    'Latitud Antena': 'lat_antenna', 
    'Longitud Antena': 'lng_antenna', 
    'Azimuth Efectivo': 'azimuth', 
    'Ancho Horiz.': 'h_beamwidth',
})
.set_index('sector_name'))

df_iga_2 = (
    df_radio_sector_raw[['Sector', 'Radio Aprox. Sector [m]']]
).rename(columns={
    'Sector': 'sector_name',   
    'Radio Aprox. Sector [m]': 'sector_radius_m',   
}).set_index('sector_name')

df_iga = df_iga_1.join(df_iga_2)
display(df_iga)

,site_name,site_status,lat_antenna,lng_antenna,azimuth,h_beamwidth,sector_radius_m
sector_name,,,,,,,
XRCESEL1G,A01-E133,Operativo,-33.142181,-64.348739,39,73,618.0
XRCESEL1H,A01-E133,Operativo,-33.142181,-64.348739,159,73,573.0
XRCESEL1I,A01-E133,Operativo,-33.142181,-64.348739,279,73,713.0
XRCESEM1G,A01-E133,No Operativo,-33.142181,-64.348739,39,73,618.0
XRCESEM1H,A01-E133,No Operativo,-33.142181,-64.348739,159,73,573.0
...,...,...,...,...,...,...,...
SBBORIU12,SPU990,No Operativo,-34.790932,-58.239595,180,64,4493.0
SBBORIU13,SPU990,No Operativo,-34.790932,-58.239595,300,64,2679.0
SBBORIV21,SPU990,No Operativo,-34.790932,-58.239595,60,64,3991.0


# Add IGA data to Traffica Insights dataframes

In [15]:
def dist_m(lat1: float, lng1: float, lat2:float, lng2: float) -> float | None:
    try:
        return geopy.distance.great_circle((lat1, lng1), (lat2, lng2)).m
    except:
        return None

In [16]:
def get_sample_footprint_radius(df_ti: pd.DataFrame, min_rsrp_dbm:float, near_samples_percent: float) -> pd.DataFrame:
    df2 = df_ti[(df_ti['rsrp'] >= min_rsrp_dbm)]
    df2 = df2.reset_index()[['sector_name', 'total_samples']].groupby('sector_name').sum().reset_index()
    data = {r[0]: {'sector_name': r[0], 'total_samples': r[1]} for r in df2.values.tolist()}
    for sector_name in data.keys():
        df_sector = df_ti[df_ti.index.isin([sector_name])].sort_values(by='dist_m', ascending=True)
        total_samples = data[sector_name]['total_samples']
        accum_samples = 0
        for r in df_sector.to_dict(orient='records'):
            accum_samples += r['total_samples']
            if 100.0 * accum_samples / total_samples >= near_samples_percent:
                data[sector_name]['footprint_radius_m'] = r['dist_m']
                data[sector_name]['footprint_samples'] = accum_samples
                break
    result = pd.DataFrame(data.values())[['sector_name', 'footprint_radius_m', 'total_samples', 'footprint_samples']].set_index('sector_name')
    return result

In [20]:
df_raw = None
df_best_serv_cell = None
df_dominant_cell = None

# Raw
if df_ti_raw is not None:
    df_raw = df_ti_raw.join(df_iga)
    # Add distance to the site in metres for each sample
    df_raw['dist_m'] = df_raw.apply(lambda r: dist_m(r['lat'], r['lng'], r['lat_antenna'], r['lng_antenna']), axis=1)
    # Add sample footprint radius
    df_raw = df_iga.join(get_sample_footprint_radius(df_raw, MIN_RSRP_DBM, NEAR_SAMPLES_PERCENT))
    df_raw['overshooting_ratio'] = df_raw.apply(lambda r: r['footprint_radius_m'] / r['sector_radius_m'], axis=1)
    # Add overshooting_ratio
    df_raw = df_raw.sort_values(by='overshooting_ratio', ascending=False)
    df_raw = df_raw[df_raw['footprint_radius_m'].notnull()]
    display(df_raw)

# Best serv cell
if df_ti_best_serv_cell is not None:
    df_best_serv_cell = df_ti_best_serv_cell.join(df_iga)
    # Add distance to the site in metres for each sample
    df_best_serv_cell['dist_m'] = df_best_serv_cell.apply(lambda r: dist_m(r['lat'], r['lng'], r['lat_antenna'], r['lng_antenna']), axis=1)
    # Add sample footprint radius
    df_best_serv_cell = df_iga.join(get_sample_footprint_radius(df_best_serv_cell, MIN_RSRP_DBM, NEAR_SAMPLES_PERCENT))
    df_best_serv_cell['overshooting_ratio'] = df_best_serv_cell.apply(lambda r: r['footprint_radius_m'] / r['sector_radius_m'], axis=1)
    # Add overshooting_ratio
    df_best_serv_cell = df_best_serv_cell.sort_values(by='overshooting_ratio', ascending=False)
    df_best_serv_cell = df_best_serv_cell[df_best_serv_cell['footprint_radius_m'].notnull()]
    display(df_best_serv_cell)

# Dominant cell
if df_ti_dominant_cell is not None:
    df_dominant_cell = df_ti_dominant_cell.join(df_iga)
    # Add distance to the site in metres for each sample
    df_dominant_cell['dist_m'] = df_dominant_cell.apply(lambda r: dist_m(r['lat'], r['lng'], r['lat_antenna'], r['lng_antenna']), axis=1)
    # Add sample footprint radius
    df_dominant_cell = df_iga.join(get_sample_footprint_radius(df_dominant_cell, MIN_RSRP_DBM, NEAR_SAMPLES_PERCENT))
    df_dominant_cell['overshooting_ratio'] = df_dominant_cell.apply(lambda r: r['footprint_radius_m'] / r['sector_radius_m'], axis=1)
    # Add overshooting_ratio
    df_dominant_cell = df_dominant_cell.sort_values(by='overshooting_ratio', ascending=False)
    df_dominant_cell = df_dominant_cell[df_dominant_cell['footprint_radius_m'].notnull()]
    display(df_dominant_cell)


,site_name,site_status,lat_antenna,lng_antenna,azimuth,h_beamwidth,sector_radius_m,footprint_radius_m,total_samples,footprint_samples,overshooting_ratio
sector_name,,,,,,,,,,,
ZSFCENN12,A07-S043,Operativo,-31.642728,-60.705930,182,68,231.0,984.549885,225.0,203.0,4.262121
ZSFCENM12,A07-S043,Operativo,-31.642728,-60.705930,180,61,231.0,896.080881,383.0,345.0,3.879138
ZALEYEM12,A07-S176,Operativo,-31.564994,-60.519569,160,61,5251.0,16431.315599,35.0,32.0,3.129178
ZSTALBO13,A07-S382,Operativo,-31.696103,-60.750395,335,70,1104.0,3406.214925,2.0,2.0,3.085340
ZSTALBM13,A07-S382,Operativo,-31.696103,-60.750395,339,62,1104.0,3376.639887,18.0,17.0,3.058551
...,...,...,...,...,...,...,...,...,...,...,...
ZSFZASL12,A07-S277,Operativo,-31.655681,-60.709449,125,66,4830.0,232.473688,543.0,489.0,0.048131
ZSJRINM13,A07-S012,Operativo,-31.604573,-60.576751,285,66,5124.0,225.620700,2.0,2.0,0.044032
ZSFZASN12,A07-S277,Operativo,-31.655681,-60.709449,123,72,7022.0,293.469081,258.0,233.0,0.041793


,site_name,site_status,lat_antenna,lng_antenna,azimuth,h_beamwidth,sector_radius_m,footprint_radius_m,total_samples,footprint_samples,overshooting_ratio
sector_name,,,,,,,,,,,
ZSFCENL12,A07-S043,Operativo,-31.642728,-60.705930,180,61,231.0,1503.819327,29.0,27.0,6.510040
ZSFCENO12,A07-S043,Operativo,-31.642728,-60.705930,182,68,231.0,1474.275861,10.0,10.0,6.382147
ZSFCENM12,A07-S043,Operativo,-31.642728,-60.705930,180,61,231.0,1344.752260,167.0,156.0,5.821438
ZSFCGOM11,A07-S390,Operativo,-31.658044,-60.710720,0,60,217.0,1122.050648,4.0,4.0,5.170740
ZSFCENN12,A07-S043,Operativo,-31.642728,-60.705930,182,68,231.0,996.326206,95.0,89.0,4.313100
...,...,...,...,...,...,...,...,...,...,...,...
ZSFZASN12,A07-S277,Operativo,-31.655681,-60.709449,123,72,7022.0,351.686840,144.0,130.0,0.050084
ZSFMDSN12,A07-S265,Operativo,-31.605795,-60.673039,92,68,4935.0,242.735594,7.0,7.0,0.049187
ZSTOMEO12,A07-S044,Operativo,-31.658808,-60.841578,219,68,7635.0,339.186614,3.0,3.0,0.044425


,site_name,site_status,lat_antenna,lng_antenna,azimuth,h_beamwidth,sector_radius_m,footprint_radius_m,total_samples,footprint_samples,overshooting_ratio
sector_name,,,,,,,,,,,
ZSFCENL12,A07-S043,Operativo,-31.642728,-60.705930,180,61,231.0,1546.251786,50.0,47.0,6.693731
ZSFCENN12,A07-S043,Operativo,-31.642728,-60.705930,182,68,231.0,1490.307322,38.0,36.0,6.451547
ZSFCENO12,A07-S043,Operativo,-31.642728,-60.705930,182,68,231.0,1474.275861,19.0,18.0,6.382147
ZSFCENF12,A07-S043,Operativo,-31.642728,-60.705930,180,62,231.0,842.710935,1114.0,1003.0,3.648099
ZSFCENM12,A07-S043,Operativo,-31.642728,-60.705930,180,61,231.0,776.101957,110.0,99.0,3.359749
...,...,...,...,...,...,...,...,...,...,...,...
ZSFJBOL11,A07-S243,Operativo,-31.587274,-60.705909,5,66,4589.0,174.780534,568.0,539.0,0.038087
ZSTOMEN12,A07-S044,Operativo,-31.658808,-60.841578,220,66,7804.0,234.046061,1.0,1.0,0.029991
ZSFCGOF11,A07-S390,Operativo,-31.658044,-60.710720,0,60,217.0,6.146522,17.0,34.0,0.028325


In [ ]:
# Save to Excel
with pd.ExcelWriter('output\\Overshooting Cell Detection.xlsx') as writer:  
    if df_raw is not None:
        df_raw.to_excel(writer, sheet_name='Raw')
    if df_best_serv_cell is not None:
        df_best_serv_cell.to_excel(writer, sheet_name='Best Serv Cell')
    if df_dominant_cell is not None:
        df_dominant_cell.to_excel(writer, sheet_name='Dominant Cell')